In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import climetlab as cml
import climetlab_s2s_ai_challenge

In [ ]:
def fix_dataset_dims(d):
    day_of_year = d.forecast_time[0].dt.dayofyear.data.item()

    new_d = d.expand_dims("forecast_dayofyear").assign_coords(
        forecast_dayofyear=[day_of_year]
    )
    new_d = new_d.assign_coords(forecast_year=new_d.forecast_time.dt.year).swap_dims(
        forecast_time="forecast_year"
    )

    dims = set(new_d.dims)
    dims.difference_update(
        ("forecast_dayofyear", "forecast_year", "latitude", "longitude")
    )

    new_d = new_d.transpose(
        "forecast_year", "forecast_dayofyear", *dims, "latitude", "longitude"
    )

    return new_d

# First surprise

In [ ]:
cmlds = cml.load_dataset("s2s-ai-challenge-training-output-reference",
                         date = 20200220,
                         parameter='tp'                         
                       )

xrds = fix_dataset_dims(cmlds.to_xarray())

In [ ]:
xrds.sel(forecast_year=2012).isnull().sum(dim=['latitude', 'longitude']).tp.plot()

Numer of null values increases after a certain lead time. 
I think there are many cases of this in the dataset.

Here is an example of correct lead time.

In [ ]:
xrds.sel(forecast_year=2012).isel(lead_time=5).tp.plot()

Lead time 20 has only nulls.

In [ ]:
xrds.sel(forecast_year=2012).isel(lead_time=20).tp.plot()

# Second surprise

In [ ]:
cmlds = cml.load_dataset("s2s-ai-challenge-training-output-reference",
                         date = 20200326,
                         parameter='tp'                         
                       )

xrds = fix_dataset_dims(cmlds.to_xarray())

In [ ]:
xrds.forecast_dayofyear

In [ ]:
xrds.isel(lead_time=1).sel(forecast_year=2015, forecast_dayofyear=86).tp.plot()

Note the gap inside of Africa. 
I saw different gaps of differents sizes in that same region.